# Task 5: Strategy Backtesting

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

# Load the cleaned data
tsla_df = pd.read_csv('../data/TSLA_cleaned.csv', index_col='Date', parse_dates=True)
bnd_df = pd.read_csv('../data/BND_cleaned.csv', index_col='Date', parse_dates=True)
spy_df = pd.read_csv('../data/SPY_cleaned.csv', index_col='Date', parse_dates=True)

## 1. Define Backtesting Period and Benchmark

In [ ]:
backtest_start_date = '2024-08-01'
backtest_end_date = '2025-07-31'

benchmark_weights = {'SPY': 0.6, 'BND': 0.4}

## 2. Get the Optimal Portfolio Weights from Task 4

In [ ]:
# We need to re-calculate the optimal weights based on the data available before the backtesting period
train_data_tsla = tsla_df[:backtest_start_date]['Close']
train_data_bnd = bnd_df[:backtest_start_date]['Close']
train_data_spy = spy_df[:backtest_start_date]['Close']

# Expected returns
stepwise_fit = auto_arima(train_data_tsla, trace=False, suppress_warnings=True)
p, d, q = stepwise_fit.order
model = ARIMA(train_data_tsla, order=(p, d, q))
model_fit = model.fit()
forecast = model_fit.forecast(steps=252)
expected_return_tsla = (forecast.iloc[-1] - train_data_tsla.iloc[-1]) / train_data_tsla.iloc[-1]
expected_return_bnd = train_data_bnd.pct_change().mean() * 252
expected_return_spy = train_data_spy.pct_change().mean() * 252
expected_returns = np.array([expected_return_tsla, expected_return_bnd, expected_return_spy])

# Covariance matrix
returns_tsla = train_data_tsla.pct_change().dropna()
returns_bnd = train_data_bnd.pct_change().dropna()
returns_spy = train_data_spy.pct_change().dropna()
returns_df = pd.concat([returns_tsla, returns_bnd, returns_spy], axis=1).dropna()
returns_df.columns = ['TSLA', 'BND', 'SPY']
cov_matrix = returns_df.cov() * 252

# Monte Carlo simulation to find the max Sharpe ratio portfolio
num_portfolios = 10000
results = np.zeros((3, num_portfolios))
weights_record = []
for i in range(num_portfolios):
    weights = np.random.random(3)
    weights /= np.sum(weights)
    weights_record.append(weights)
    portfolio_return = np.sum(weights * expected_returns)
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    results[0,i] = portfolio_return
    results[1,i] = portfolio_stddev
    results[2,i] = (portfolio_return - 0.02) / portfolio_stddev

results_frame = pd.DataFrame(results.T, columns=['ret','stdev','sharpe'])
results_frame['weights'] = weights_record
max_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]
optimal_weights = max_sharpe_port['weights']

## 3. Simulate the Strategy and Benchmark

In [ ]:
# Get the returns for the backtesting period
backtest_returns_tsla = tsla_df[backtest_start_date:backtest_end_date]['Close'].pct_change().dropna()
backtest_returns_bnd = bnd_df[backtest_start_date:backtest_end_date]['Close'].pct_change().dropna()
backtest_returns_spy = spy_df[backtest_start_date:backtest_end_date]['Close'].pct_change().dropna()
backtest_returns = pd.concat([backtest_returns_tsla, backtest_returns_bnd, backtest_returns_spy], axis=1).dropna()
backtest_returns.columns = ['TSLA', 'BND', 'SPY']

# Strategy portfolio returns
strategy_returns = (backtest_returns * optimal_weights).sum(axis=1)

# Benchmark portfolio returns
benchmark_returns = (backtest_returns[['SPY', 'BND']] * pd.Series(benchmark_weights)).sum(axis=1)

## 4. Analyze Performance

In [ ]:
# Calculate cumulative returns
strategy_cumulative_returns = (1 + strategy_returns).cumprod() - 1
benchmark_cumulative_returns = (1 + benchmark_returns).cumprod() - 1

In [ ]:
# Plot the cumulative returns
plt.figure(figsize=(14, 7))
plt.plot(strategy_cumulative_returns, label='Optimized Strategy')
plt.plot(benchmark_cumulative_returns, label='60/40 Benchmark')
plt.title('Strategy Backtest vs. Benchmark')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()

In [ ]:
# Calculate performance metrics
total_return_strategy = strategy_cumulative_returns.iloc[-1]
total_return_benchmark = benchmark_cumulative_returns.iloc[-1]
sharpe_ratio_strategy = (strategy_returns.mean() * 252 - 0.02) / (strategy_returns.std() * np.sqrt(252))
sharpe_ratio_benchmark = (benchmark_returns.mean() * 252 - 0.02) / (benchmark_returns.std() * np.sqrt(252))

print("Optimized Strategy Performance:")
print(f'Total Return: {total_return_strategy:.2%}')
print(f'Sharpe Ratio: {sharpe_ratio_strategy:.2f}')

print("Benchmark Performance:")
print(f'Total Return: {total_return_benchmark:.2%}')
print(f'Sharpe Ratio: {sharpe_ratio_benchmark:.2f}')

## 5. Conclusion

The backtest results show that the optimized strategy [outperformed/underperformed] the 60/40 benchmark portfolio during the backtesting period. The strategy yielded a total return of [X]% and a Sharpe ratio of [Y], compared to the benchmark's total return of [A]% and Sharpe ratio of [B].

This initial backtest suggests that the model-driven approach [is/is not] a viable strategy for enhancing portfolio returns. However, it is important to note that this is a simplified backtest, and further analysis with more sophisticated techniques, such as rebalancing and transaction costs, would be necessary to validate these findings.